In [ ]:
%cd /content/drive/MyDrive/FDP Feb 2022/practice

/content/drive/MyDrive/FDP Feb 2022/practice


In [ ]:
from torchvision import transforms
data_transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), 
                    transforms.Normalize(
                      mean=[0.5,0.5, 0.5], std=[0.5,0.5,0.5])
                    ])

In [ ]:
!ls

model.h5  satellite_data  satellite_data.zip  split_data  weights.h5


In [ ]:
import torchvision
train_data = torchvision.datasets.ImageFolder(root='split_data/train', transform= data_transform)

In [ ]:
valid_data = torchvision.datasets.ImageFolder(root='split_data/val', transform= data_transform)
test_data = torchvision.datasets.ImageFolder(root='split_data/test', transform= data_transform)

In [ ]:
import torch
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True, pin_memory=True)

In [ ]:
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=8, shuffle=False, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=False, pin_memory=True)

In [ ]:
class CNN_model(torch.nn.Module):
  def __init__(self):
    super(CNN_model, self).__init__()
    self.conv1 = torch.nn.Conv2d(3, 16, 3)
    self.conv2 = torch.nn.Conv2d(16, 32, 3)
    self.fc1 = torch.nn.Linear(32*110*110, 16)
    self.fc2 = torch.nn.Linear(16,2)

  def forward(self, x):
    x = self.conv1(x)
    x = torch.nn.functional.relu(x)
    x = self.conv2(x)
    x = torch.nn.functional.relu(x)
    x = torch.nn.functional.max_pool2d(x, 2)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = torch.nn.functional.relu(x)
    x = self.fc2(x)
    output = torch.nn.functional.log_softmax(x, dim=1)
    return output

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
cnnmodel = CNN_model().to(device)
print(cnnmodel)

CNN_model(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=387200, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=2, bias=True)
)


In [ ]:
optimizer = torch.optim.Adam(cnnmodel.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def train(model, trainloader, optimizer, criterion):
  model.train()
  train_loss = 0.0
  train_correct = 0
  count = 0
  for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
    count = count +1
    image, label = data
    image = image.to(device)
    label = label.to(device)

    optimizer.zero_grad()
    output = model(image)
    loss = criterion(output, label)
    train_loss += loss.item()

    _, pred = torch.max(output.data, 1)
    train_correct = train_correct+ (pred == label).sum().item()

    loss.backward()

    optimizer.step()

  epoch_loss = train_loss/ count
  epoch_acc = 100 * (train_correct / len(trainloader.dataset))
  return epoch_loss, epoch_acc

In [ ]:
def validation(model, validloader, criterion):
  model.eval()
  val_loss = 0.0
  val_correct = 0
  count = 0
  with torch.no_grad():
    for i, data in tqdm(enumerate(validloader), total=len(validloader)):
      count = count +1
      image, label = data
      image = image.to(device)
      label = label.to(device)

      output = model(image)
      loss = criterion(output, label)
      
      val_loss += loss.item()

      _, pred = torch.max(output.data, 1)
      val_correct = val_correct+ (pred == label).sum().item()
    
  epoch_loss = val_loss/ count
  epoch_acc = 100 * (val_correct / len(validloader.dataset))
  return epoch_loss, epoch_acc

In [ ]:
from tqdm.auto import tqdm
import time
epochs = 5
train_loss, train_acc, val_loss, val_acc = [], [], [], []

for epoch in range(epochs):
  print("Epoch: ", epoch+1)
  train_epoch_loss, train_epoch_acc = train(cnnmodel, train_loader, optimizer, criterion)
  val_epoch_loss, val_epoch_acc = validation(cnnmodel, train_loader, criterion)
  train_loss.append(train_epoch_loss)
  train_acc.append(train_epoch_acc)
  val_loss.append(val_epoch_loss)
  val_acc.append(val_epoch_acc)

  print("Training loss: ",train_epoch_loss, "Training accuracy", train_epoch_acc)
  print("Validation loss: ",val_loss, "Validation accuracy", val_acc)

  print('-'*50)
  time.sleep(5)

Epoch:  1


  0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/41 [00:00<?, ?it/s]

Training loss:  0.15361067323332153 Training accuracy 94.75308641975309
Validation loss:  [0.07572381350597958] Validation accuracy [98.14814814814815]
--------------------------------------------------
Epoch:  2


  0%|          | 0/41 [00:00<?, ?it/s]

...

  0%|          | 0/41 [00:00<?, ?it/s]

Training loss:  0.06531474140838993 Training accuracy 97.22222222222221
Validation loss:  [0.07572381350597958, 0.04541224468408561, 0.04349526319625822, 0.05352988334730404] Validation accuracy [98.14814814814815, 99.38271604938271, 98.14814814814815, 97.22222222222221]
--------------------------------------------------
Epoch:  5


  0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/41 [00:00<?, ?it/s]

Training loss:  0.04056912483633269 Training accuracy 98.14814814814815
Validation loss:  [0.07572381350597958, 0.04541224468408561, 0.04349526319625822, 0.05352988334730404, 0.009054174200239842] Validation accuracy [98.14814814814815, 99.38271604938271, 98.14814814814815, 97.22222222222221, 100.0]
--------------------------------------------------
